# Tabular Playground Series - Feb 2021

## Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## Read datasets

In [2]:
train = pd.read_csv('train.zip', index_col='id')
display(train.sample(2))

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
id,,,,,,,,,,,,,,,,,,,,,
141441,B,A,A,C,B,B,A,E,C,K,...,0.449134,0.603255,0.318544,0.497725,0.556856,0.716278,0.978799,0.304839,0.419665,7.695971
25261,A,A,A,A,B,D,A,E,E,F,...,0.781307,0.660872,0.875735,0.452345,0.414665,0.392382,0.845341,0.894648,0.656414,7.585726


In [3]:
test = pd.read_csv('test.zip', index_col='id')
display(test.sample(2))

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
id,,,,,,,,,,,,,,,,,,,,,
283917,A,B,A,D,B,B,A,E,C,F,...,0.416447,0.320897,0.409647,0.301215,0.210705,0.222360,0.418633,0.860642,0.352411,0.809922
86210,A,A,A,C,B,B,A,E,C,H,...,0.813192,0.354757,0.510837,0.299033,0.393696,0.530452,0.399905,0.416512,0.344361,0.749557


In [4]:
submission = pd.read_csv('sample_submission.csv', index_col='id')
display(submission.sample(2))

,target
id,
426134,0.5
216473,0.5


In [5]:
X = train.drop(['target'], axis=1)
y = train.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state = 42)

## EDA

## LazyPredict

In [ ]:
!pip install -U -q pip
!pip install -U -q setuptools
!pip install -q lazypredict
from lazypredict.Supervised import LazyRegressor

In [ ]:
reg = LazyRegressor(verbose=0, ignore_warnings=True)
models,predictions = reg.fit(X_train, X_test, y_train, y_test)

In [ ]:
print(models)

## EvalML

In [ ]:
!pip install -U -q pip
!pip install -U -q setuptools
!pip install -q evalml
import evalml
from evalml.automl import AutoMLSearch

In [ ]:
X_train, X_test, y_train, y_test = evalml.preprocessing.split_data(X, y, problem_type='regression')

In [ ]:
automl = AutoMLSearch(X_train=X_train, y_train=y_train, problem_type='regression')
automl.search()

In [ ]:
automl.rankings

In [ ]:
automl.describe_pipeline(automl.rankings.iloc[0]["id"])

In [ ]:
best_pipeline.score(X_test, y_test)

## AutoGluon

In [6]:
!pip install -U -q pip
!pip install -U -q setuptools
!pip install -U -q "mxnet<2.0.0"
!pip install -q autogluon

  ERROR: Command errored out with exit status 1:
   command: /opt/conda/bin/python /home/vscode/.local/lib/python3.8/site-packages/pip/_vendor/pep517/_in_process.py build_wheel /tmp/tmphwerjw9v
       cwd: /tmp/pip-install-vh7kssut/configspace_d52c7a21e8d540ee9c01a603f020e251
  Complete output (45 lines):
  /tmp/pip-build-env-_njjg1kr/overlay/lib/python3.8/site-packages/setuptools/dist.py:642: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
    warnings.warn(
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.linux-x86_64-3.8
  creating build/lib.linux-x86_64-3.8/ConfigSpace
  copying ConfigSpace/exceptions.py -> build/lib.linux-x86_64-3.8/ConfigSpace
  copying ConfigSpace/__version__.py -> build/lib.linux-x86_64-3.8/ConfigSpace
  copying ConfigSpace/__init__.py -> build/lib.linux-x86_64-3.8/ConfigSpace
  creating build/lib.linux-x86_64

In [7]:
from autogluon.tabular import TabularPredictor

ModuleNotFoundError: No module named 'autogluon'

In [ ]:
predictor = TabularPredictor(label='target', 
                             problem_type='regression', 
                             eval_metric='root_mean_squared_error'
                             ).fit(train, 
                                   #time_limit=840, 
                                   presets='high_quality_fast_inference_only_refit'
                                   )

In [ ]:
predictor.leaderboard()

In [ ]:
submission['target'] = predictor.predict(test)
submission.to_csv('../output/submission_autogluon.csv')

## PyCaret